en terminal
pip install watchdog


In [ ]:
import os
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time

# Configurar Firebase con las credenciales
cred = credentials.Certificate("firebase-key.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

# Ruta a la carpeta donde están los archivos CSV
csv_folder = "C:/Users/santi/Desktop/PI/archive/csv"  # Cambiar según sea necesario

# Función para subir datos a Firebase
def upload_to_firebase(csv_file):
    full_csv_path = os.path.join(csv_folder, csv_file)

    try:
        df = pd.read_csv(full_csv_path)

        # Crear o referenciar la colección en Firestore
        collection_name = os.path.splitext(csv_file)[0]
        collection_ref = db.collection(collection_name)

        # Subir los datos a Firestore
        for index, row in df.iterrows():
            doc_ref = collection_ref.document(str(index))
            doc_ref.set(row.to_dict())

        print(f"Datos de {csv_file} actualizados en Firebase.")

    except Exception as e:
        print(f"Error al actualizar {csv_file}: {e}")

# Clase que maneja eventos de cambio en archivos CSV
class CSVHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.is_directory:
            return
        if event.src_path.endswith(".csv"):
            csv_file = os.path.basename(event.src_path)
            print(f"Detectado cambio en {csv_file}, actualizando en Firebase...")
            upload_to_firebase(csv_file)

# Configurar el monitoreo de la carpeta
event_handler = CSVHandler()
observer = Observer()
observer.schedule(event_handler, path=csv_folder, recursive=False)

print("Monitoreo iniciado... (Presiona Ctrl+C para detener)")
observer.start()

# Mantener el script en ejecución
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
    print("Monitoreo detenido.")

observer.join()


Task Scheduler en Windows
1- Abrir el Programador de Tareas
Presiona Win + R, escribe taskschd.msc y presiona Enter.

2- Crear una nueva tarea
En el panel derecho, selecciona "Crear una tarea básica...".

3-Configurar la tarea
Nombre: Actualizar CSV Firebase
Descripción: Ejecuta el script de actualización de Firebase a las 10 AM.

 4- Establecer la frecuencia

Selecciona "Diariamente" y configura la hora:
Hora de inicio: 10:00 AM
Repetir cada: 1 día
Click en Siguiente.

5- Seleccionar la acción
Elige "Iniciar un programa" y haz click en Siguiente.
Indicar el programa a ejecutar

En Programa/script, escribe la ruta de Python, por ejemplo:
C:\Users\santi\AppData\Local\Programs\Python\Python310\python.exe
"C:\Users\santi\Desktop\PI\monitor_csv.py"
